In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from urllib.request import urlopen, Request
from selenium.webdriver.support import expected_conditions as EC

In [2]:
driver = webdriver.Chrome()
url = 'https://th.investing.com/earnings-calendar/'
driver.get(url)

# Try to close popup if it appears
try:
    popup_close = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".popupCloseIcon.largeBannerCloser"))
    )
    popup_close.click()
    print("Found and clicked the 'Close' button!")
except Exception as e:
    print("Popup close button not found or not clickable:", e)

# Click the Filters button
try:
    filters_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "filterStateAnchor"))
    )
    filters_button.click()
    print("Found and clicked the 'Filters' button!")
except Exception as e:
    print("Filters button not found or not clickable:", e)

# Click the Clear button
try:
    erase_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.LINK_TEXT, "ล้างข้อมูล"))
    )
    erase_button.click()
    print("Found and clicked the 'Clear' button!")
except Exception as e:
    print("Clear button not found or not clickable:", e)

# Click the United States button
try:
    country_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "country5"))
    )
    country_button.click()
    print("Found and clicked the 'United States' button!")
except Exception as e:
    print("United States button not found or not clickable:", e)

# Click the 2 Star button
try:
    star2_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "importance2"))
    )
    star2_button.click()
    print("Found and clicked the '2 Star' button!")
except Exception as e:
    print("2 Star button not found or not clickable:", e)

# Click the 3 Star button
try:
    star3_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "importance3"))
    )
    star3_button.click()
    print("Found and clicked the '3 Star' button!")
except Exception as e:
    print("3 Star button not found or not clickable:", e)

# Click the Apply button
try:
    apply_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "ecSubmitButton"))
    )
    apply_button.click()
    print("Found and clicked the 'Apply' button!")
except Exception as e:
    print("Apply button not found or not clickable:", e)


time.sleep(5)   
response = driver.page_source
soup = BeautifulSoup(response, 'html.parser')
table = soup.find_all("tbody")
driver.quit()

Popup close button not found or not clickable: Message: 

Found and clicked the 'Filters' button!
Found and clicked the 'Clear' button!
Found and clicked the 'United States' button!
Found and clicked the '2 Star' button!
Found and clicked the '3 Star' button!
Found and clicked the 'Apply' button!


In [3]:
def earnings_calendar(table_bodies):
    data = []
    current_date = None

    for tbody in table_bodies:
        for tr in tbody.find_all("tr"):
            date_td = tr.find(class_="theDay")
            if date_td:
                current_date = date_td.text.strip()
            else:
                company_span = tr.find(class_="earnCalCompanyName middle")
                ticker_a = tr.find(class_="bold middle")
                all_left_strong_tds = tr.find_all(class_="leftStrong")
                market_cap = tr.find(class_="right")
                if tr.find(class_="marketOpen genToolTip oneliner reverseToolTip"):
                    outtime_status = "Before Market"
                elif tr.find(class_="marketClosed genToolTip oneliner reverseToolTip"):
                    outtime_status = "After Market"
                else:
                    outtime_status = "N/A"
                estimate_eps = 'N/A'
                estimate_revenue = 'N/A'
                
                if company_span and ticker_a:
                    company = company_span.text.strip() if company_span else None
                    ticker = ticker_a.text.strip() if ticker_a else None
                    estimate_eps = all_left_strong_tds[0].text.replace('/','').strip() if estimate_eps else None
                    estimate_revenue = all_left_strong_tds[1].text.replace('/','').strip() if estimate_revenue else None
                    market_cap = market_cap.text.strip() if market_cap else None
                    
                    date=datetime.strptime(current_date, '%A, %B %d, %Y')
                    
                    data.append({
                        'date': date,
                        'company': company,
                        'ticker': ticker,
                        'estimate_eps': estimate_eps,
                        'estimate_revenue': estimate_revenue,
                        'market_cap': market_cap,
                        'outtime': outtime_status
                    })
                else:
                    continue
    return data

In [4]:
earnings_calendar_data = earnings_calendar(table)
df = pd.DataFrame(earnings_calendar_data)

In [5]:
df.head(20)

,date,company,ticker,estimate_eps,estimate_revenue,market_cap,outtime
0,2025-05-27,PDD Holdings DRC,PDD,19.44,102.98B,169.28B,Before Market
1,2025-05-28,NVIDIA,NVDA,0.8925,43.12B,3.2T,After Market
2,2025-05-28,Salesforce Inc,CRM,2.54,9.75B,262.06B,After Market
3,2025-05-28,Synopsys,SNPS,3.39,1.6B,77.13B,After Market
4,2025-05-28,Agilent Technologies,A,1.27,1.63B,30.94B,After Market
5,2025-05-28,HP Inc,HPQ,0.7927,13.07B,26.38B,After Market
6,2025-05-28,Nordson,NDSN,2.36,672.95M,11.01B,After Market
7,2025-05-29,Dell Tech,DELL,1.69,23.14B,76.62B,After Market
8,2025-05-29,Marvell,MRVL,0.612,1.88B,52.43B,After Market
9,2025-05-29,Zscaler,ZS,0.7564,666.4M,39.32B,After Market


In [7]:
now = datetime.now()
date_time = now.strftime("%Y-%m-%d %H-%M-%S").strip().replace(' ', '_')
df.to_csv(f'Data/Earnings_Calendar/{date_time}.csv', index=False)